In [1]:
import logging
import re
from typing import Optional

In [2]:
logger = logging.getLogger(__name__)


def update_utility_classes(
    current_classes: str,
    remove_classes: Optional[list[str]] = None,
    add_classes: Optional[list[str]] = None,
    ignore_prefix_warning: bool = False,
) -> str:
    """Update a utility class string by removing and/or adding classes.

    Removes/adds utility classes from/to the `current_classes` string. At least one of
    the arguments `remove_classes` or `add_classes` must be provided. These arguments
    are given in the form of a list of strings representing the classes to be removed or
    added.

    If an incoming add class has a prefix that matches a class(es) in the
    `current_classes` string, a warning will be shown but the add will still be
    completed. The warning is there to make it known that the addition of the class
    could override these prefix matched existing classes in the `current_classes`
    string. The developer should evaluate the add and ensure that this is not the case.
    It is possible that there is no overriding behavior even when there are multiple
    classes with the same prefix. If this is the case, the warning can be suppressed by
    setting the `ignore_prefix_warning` to True.

    Parameters
    ----------
    current_classes : str
        Current utility class string.
    remove_classes : Optional[list[str]], optional
        Classes to be removed, by default None.
    add_classes : Optional[list[str]], optional
        Classes to be added, by default None.
    ignore_prefix_warning : bool, optional
        Flag for suppressing the prefix warning, by default False.

    Returns
    -------
    str
        Updated utility class string.
    """
    # Check if both optional arguments were not provided.
    if remove_classes is None and add_classes is None:
        raise RuntimeError(
            "Both arguments `remove_classes` and `add_classes` were not provided. "
            "Please provide at least one of these arguments."
        )

    current_class_list = current_classes.split()
    # Pattern to match the prefix of a utility class.
    prefix_pattern = r"^-?([a-z:]+)"

    # Remove and/or Add Classes --------------------------------------------------------
    if remove_classes:
        for remove_class in remove_classes:
            try:
                current_class_list.remove(remove_class)
            except ValueError as err:
                raise ValueError(
                    f"The string '{remove_class}', from the `remove_classes` argument, "
                    f"was not found in the `current_classes` string:\n"
                    f"'{current_classes}'"
                ) from err

    if add_classes:
        for add_class in add_classes:
            # Check if the class is already in the `current_class_list`.
            if add_class in current_class_list:
                raise RuntimeError(
                    f"The string '{add_class}', from the `add_classes` argument, is "
                    f"already found within the `current_classes` string:\n"
                    f"'{current_classes}'"
                )

            # Capture the prefix of the incoming add class.
            try:
                prefix = re.search(prefix_pattern, add_class).group(1)
            except AttributeError as err:
                raise RuntimeError(
                    f"The string '{add_class}', from the `add_classes` argument, is "
                    f"not a valid utility class."
                ) from err

            # Capture all classes from `current_classes_list` that match the prefix for
            # the current add class.
            prefix_match_classes = []
            for util_class in current_class_list:
                prefix_match = re.search(prefix, util_class)
                if prefix_match:
                    prefix_match_classes.append(util_class)

            # Warn that there are matches that could result in classes that get
            # overridden by the add. This is a warning and not an error because it is
            # possible to have more than one utility class with the same prefix and not
            # have any overriding behavior.
            if prefix_match_classes:
                if not ignore_prefix_warning:
                    logger.warning(
                        f"WARNING: Upon adding the string '{add_class}', the following "
                        f"class(es) with the same prefix '{prefix}' were found within "
                        f"the `current_classes` string: {prefix_match_classes}\nIf "
                        f"this addition does not result in conflicts, this warning can "
                        f"be suppressed by setting the `ignore_prefix_warning` "
                        f"argument to True.\n"
                    )
            current_class_list.append(add_class)
    return " ".join(current_class_list)

### Tests

#### `current_classes` Test String

In [3]:
current_classes_example = (
    "px-4 py-2 flex border-4 border-opacity-30 border-black "
    "bg-slate-800 hover:bg-slate-700 focus:text-slate-800 "
    "hover:sm:text-slate-300"
)
current_classes_example

'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300'

#### Test Warning/Error Conditions

In [4]:
# Both `remove_classes` and `add_classes` arguments not provided.
update_utility_classes(
    current_classes=current_classes_example,
)

RuntimeError: Both arguments `remove_classes` and `add_classes` were not provided. Please provide at least one of these arguments.

In [5]:
# Try to remove a class that is not in `current_classes`.
update_utility_classes(
    current_classes=current_classes_example,
    remove_classes=["pt-8"],
)

ValueError: The string 'pt-8', from the `remove_classes` argument, was not found in the `current_classes` string:
'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300'

In [6]:
# Try a duplicate add class.
update_utility_classes(
    current_classes=current_classes_example,
    add_classes=["px-4"],
)

RuntimeError: The string 'px-4', from the `add_classes` argument, is already found within the `current_classes` string:
'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300'

In [7]:
# Try to add an invalid utility class.
update_utility_classes(
    current_classes=current_classes_example,
    add_classes=["1"],
)

RuntimeError: The string '1', from the `add_classes` argument, is not a valid utility class.

In [8]:
# Try a class whose prefix is matched by some of the classes in `current_classes`.
update_utility_classes(
    current_classes=current_classes_example,
    add_classes=["border-gray-700"],
)

If this addition does not result in conflicts, this warning can be suppressed by setting the `ignore_prefix_warning` argument to True.



'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 border-gray-700'

In [9]:
# Try multiple classes whose prefixes are matched by some of the classes in
# `current_classes`. Looking for multiple warnings to show.
update_utility_classes(
    current_classes=current_classes_example,
    add_classes=["border-gray-700", "py-8", "-inset-1"],
)

If this addition does not result in conflicts, this warning can be suppressed by setting the `ignore_prefix_warning` argument to True.

If this addition does not result in conflicts, this warning can be suppressed by setting the `ignore_prefix_warning` argument to True.



'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 border-gray-700 py-8 -inset-1'

In [10]:
# Suppress the warning.
update_utility_classes(
    current_classes=current_classes_example,
    add_classes=["border-gray-700", "py-8", "-inset-1"],
    ignore_prefix_warning=True,
)

'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 border-gray-700 py-8 -inset-1'

#### Test Expected Input Conditions

In [11]:
# Try removes.
rm_cls1 = ["py-2", "hover:bg-slate-700"]
rm_cls2 = []

rm1 = update_utility_classes(
    current_classes=current_classes_example,
    remove_classes=rm_cls1,
)
rm2 = update_utility_classes(
    current_classes=current_classes_example,
    remove_classes=rm_cls2,
)

print(f"`current_classes`: '{current_classes_example}'\n")

print(f"`remove_classes`: {rm_cls1}\n" f"return value:\n'{rm1}'\n")

print(
    f"`remove_classes`: {rm_cls2}\n"
    f"no change in `current_classes`: {rm2 == current_classes_example}\n"
)

`current_classes`: 'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300'

`remove_classes`: ['py-2', 'hover:bg-slate-700']
return value:
'px-4 flex border-4 border-opacity-30 border-black bg-slate-800 focus:text-slate-800 hover:sm:text-slate-300'

`remove_classes`: []
no change in `current_classes`: True



In [12]:
# Try adds.
add_cls1 = ["pt-8", "columns-4"]
add_cls2 = []

add1 = update_utility_classes(
    current_classes=current_classes_example,
    add_classes=add_cls1,
)
add2 = update_utility_classes(
    current_classes=current_classes_example,
    add_classes=add_cls2,
)

print(f"`current_classes`: '{current_classes_example}'\n")

print(f"`add_classes`: {add_cls1}\n" f"return value:\n'{add1}'\n")

print(
    f"`add_classes`: {add_cls2}\n"
    f"no change in `current_classes`: {add2 == current_classes_example}\n"
)

`current_classes`: 'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300'

`add_classes`: ['pt-8', 'columns-4']
return value:
'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 pt-8 columns-4'

`add_classes`: []
no change in `current_classes`: True



In [13]:
# Try both removes and adds.
add_cls1 = ["pt-8", "columns-4"]
add_cls2 = []
rm_cls1 = ["py-2", "hover:bg-slate-700"]
rm_cls2 = []

both1 = update_utility_classes(
    current_classes=current_classes_example,
    remove_classes=rm_cls1,
    add_classes=add_cls1,
)
both2 = update_utility_classes(
    current_classes=current_classes_example,
    remove_classes=rm_cls2,
    add_classes=add_cls1,
)
both3 = update_utility_classes(
    current_classes=current_classes_example,
    remove_classes=rm_cls1,
    add_classes=add_cls2,
)
both4 = update_utility_classes(
    current_classes=current_classes_example,
    remove_classes=rm_cls2,
    add_classes=add_cls2,
)

print(f"`current_classes`: '{current_classes_example}'\n")

print(
    f"`remove_classes`: {rm_cls1}\n"
    f"`add_classes`: {add_cls1}\n"
    f"return value:\n'{both1}'\n"
)

print(
    f"`remove_classes`: {rm_cls2}\n"
    f"`add_classes`: {add_cls1}\n"
    f"return value:\n'{both2}'\n"
)

print(
    f"`remove_classes`: {rm_cls1}\n"
    f"`add_classes`: {add_cls2}\n"
    f"return value:\n'{both3}'\n"
)

print(
    f"`remove_classes`: {rm_cls2}\n"
    f"`add_classes`: {add_cls2}\n"
    f"no change in `current_classes`: {both4 == current_classes_example}\n"
)

`current_classes`: 'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300'

`remove_classes`: ['py-2', 'hover:bg-slate-700']
`add_classes`: ['pt-8', 'columns-4']
return value:
'px-4 flex border-4 border-opacity-30 border-black bg-slate-800 focus:text-slate-800 hover:sm:text-slate-300 pt-8 columns-4'

`remove_classes`: []
`add_classes`: ['pt-8', 'columns-4']
return value:
'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 pt-8 columns-4'

`remove_classes`: ['py-2', 'hover:bg-slate-700']
`add_classes`: []
return value:
'px-4 flex border-4 border-opacity-30 border-black bg-slate-800 focus:text-slate-800 hover:sm:text-slate-300'

`remove_classes`: []
`add_classes`: []
no change in `current_classes`: True



In [14]:
# Try adding classes with a leading '-'.
ex1 = current_classes_example + " -inset-1"
ex2 = current_classes_example + " inset-3"


add_cls1 = ["inset-3"]
add_cls2 = ["-inset-1"]
add_cls3 = ["hover:sm:text-left"]

both1 = update_utility_classes(
    current_classes=ex1,
    add_classes=add_cls1,
)
both2 = update_utility_classes(
    current_classes=ex2,
    add_classes=add_cls2,
)
both3 = update_utility_classes(
    current_classes=current_classes_example,
    add_classes=add_cls3,
    # ignore_prefix_warning=True,
)

print(f"`current_classes`: '{current_classes_example}'\n")

print(f"`add_classes`: {add_cls1}\n" f"return value:\n'{both1}'\n")

print(f"`add_classes`: {add_cls2}\n" f"return value:\n'{both2}'\n")

print(f"`add_classes`: {add_cls3}\n" f"return value:\n'{both3}'\n")

If this addition does not result in conflicts, this warning can be suppressed by setting the `ignore_prefix_warning` argument to True.

If this addition does not result in conflicts, this warning can be suppressed by setting the `ignore_prefix_warning` argument to True.

If this addition does not result in conflicts, this warning can be suppressed by setting the `ignore_prefix_warning` argument to True.



`current_classes`: 'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300'

`add_classes`: ['inset-3']
return value:
'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 -inset-1 inset-3'

`add_classes`: ['-inset-1']
return value:
'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 inset-3 -inset-1'

`add_classes`: ['hover:sm:text-left']
return value:
'px-4 py-2 flex border-4 border-opacity-30 border-black bg-slate-800 hover:bg-slate-700 focus:text-slate-800 hover:sm:text-slate-300 hover:sm:text-left'



In [15]:
# Prefixes that are not currently captured correctly.
# Prefix captured: 'motion', correct prefix: 'motion-safe:hover:-translate'
ex1 = "motion-safe:hover:-translate-x-0.5"
# Prefix captured: 'supports', correct prefix: 'supports-[backdrop-filter]:bg'
ex2 = "supports-[backdrop-filter]:bg-black/25"

In [16]:
{
    "px-4": "px",
    "flex": "flex",
    "border-4": "border",
    "border-opacity-30": "border",
    "border-black": "border",
    "bg-slate-800": "bg",
    "hover:bg-slate-700": "hover:bg",
    "focus:text-slate-800": "focus:text",
    "hover:sm:text-slate-300": "hover:sm:text",
    "inset-3": "inset",
    "-inset-1": "inset",
    "motion-safe:hover:-translate-x-0.5": "motion-safe:hover:translate",
    "supports-[backdrop-filter]:bg-black/25": "supports-[backdrop-filter]:bg",
}

{'px-4': 'px',
 'flex': 'flex',
 'border-4': 'border',
 'border-opacity-30': 'border',
 'border-black': 'border',
 'bg-slate-800': 'bg',
 'hover:bg-slate-700': 'hover:bg',
 'focus:text-slate-800': 'focus:text',
 'hover:sm:text-slate-300': 'hover:sm:text',
 'inset-3': 'inset',
 '-inset-1': 'inset',
 'motion-safe:hover:-translate-x-0.5': 'motion-safe:hover:translate',
 'supports-[backdrop-filter]:bg-black/25': 'supports-[backdrop-filter]:bg'}